In [ ]:
import pandas as pd

def analyze_convergence(csv_path, improvement_threshold=0.01, patience=2):
    """
    Analyzes training convergence statistics from a CSV log.

    Args:
        csv_path (str): Path to CSV file with at least columns:
            ['round' or 'Epoch', 'accuracy', 'loss', 'time_train_record']
        improvement_threshold (float): Min relative improvement to consider progress (default: 1%)
        patience (int): Consecutive rounds below threshold = convergence
    """

    df = pd.read_csv(csv_path)
    # Normalize column names
    df.columns = [c.strip().lower() for c in df.columns]

    # Identify key columns automatically
    acc_col = 'accuracy' if 'accuracy' in df.columns else 'eval_acc'
    loss_col = 'loss' if 'loss' in df.columns else 'eval_loss'
    time_col = 'time_train_record' if 'time_train_record' in df.columns else 'record_time_eval_and_train'
    eval_time_col = 'time_evaluate_record' if 'time_evaluate_record' in df.columns else None

    # Fill missing times with 0
    df[time_col] = df[time_col].fillna(0)
    if eval_time_col and eval_time_col in df.columns:
        df[eval_time_col] = df[eval_time_col].fillna(0)
        df["total_time"] = df[time_col] + df[eval_time_col]
    else:
        df["total_time"] = df[time_col]

    # Compute accuracy improvement per step
    df["acc_diff"] = df[acc_col].diff().abs()

    # Find convergence round
    conv_round = None
    patience_counter = 0
    for i in range(1, len(df)):
        if df.loc[i, "acc_diff"] < improvement_threshold:
            patience_counter += 1
        else:
            patience_counter = 0
        if patience_counter >= patience:
            conv_round = int(df.loc[i, df.columns[0]])
            break

    # If not found, mark as not converged
    if conv_round is None:
        conv_round = int(df.iloc[-1, 0])
        note = "⚠️ Not converged (still learning)"
    else:
        note = "✅ Converged"

    # Summaries
    acc_at_conv = df.loc[df[df.columns[0]] == conv_round, acc_col].values[0]
    loss_at_conv = df.loc[df[df.columns[0]] == conv_round, loss_col].values[0]
    time_to_conv = df.loc[df[df.columns[0]] <= conv_round, "total_time"].sum()
    avg_round_time = df["total_time"].mean()

    print("────────────────────────────────────────────")
    print(f"File: {csv_path}")
    print(f"Status: {note}")
    print(f"Convergence round/epoch: {conv_round}")
    print(f"Accuracy at convergence: {acc_at_conv:.4f}")
    print(f"Loss at convergence: {loss_at_conv:.4f}")
    print(f"Total time to convergence: {time_to_conv:.2f} s ({time_to_conv/60:.2f} min)")
    print(f"Average round time: {avg_round_time:.2f} s")
    print("────────────────────────────────────────────")

    return {
        "convergence_round": conv_round,
        "accuracy": acc_at_conv,
        "loss": loss_at_conv,
        "total_time": time_to_conv,
        "avg_round_time": avg_round_time,
        "note": note
    }


# Example usage:
if __name__ == "__main__":
    result = analyze_convergence("your_log.csv")  # <- replace with your file
    print(result)


: 